# Ocean station Papa

This notebook runs [GOTM](https://gotm.net/) with initial conditions and surface forcing at the [ocean station Papa](https://www.pmel.noaa.gov/ocs/Papa). See Fig. 4 of [Li et al., 2019](https://doi.org/10.1029/2019MS001810).

Run the case with four turbulence closure schemes:

- GLS-C01A: The generic length scale ([Umlauf and Burchard, 2003](https://doi.org/10.1357/002224003322005087)) model in the $k$-$\epsilon$ formulation with the weak-equilibrium stability function by [Canuto et al., 2001](https://doi.org/10.1175/1520-0485(2001)031%3C1413:OTPIOP%3E2.0.CO;2) (C01A).

- Three variants of KPP via [CVMix](http://cvmix.github.io):
    - KPP-CVMix ([Large et al., 1994](https://doi.org/10.1029/94RG01872), [Griffies et al., 2015](https://github.com/CVMix/CVMix-description/raw/master/cvmix.pdf))
    - KPPLT-VR12 ([Li et al., 2016](https://doi.org/10.1016%2Fj.ocemod.2015.07.020))
    - KPPLT-LF17 ([Li and Fox-Kemper, 2017](https://doi.org/10.1175%2FJPO-D-17-0085.1))
    
Run the case with a combination of vertical resolution of [1., 5., 10.] m and time step of [60, 600, 1800] s.

In [1]:
import sys
import copy
import numpy as np
import matplotlib.pyplot as plt
# add the path of gotmtool
sys.path.append("../gotmtool")
from gotmtool import *

## Create a model
Create a model with environment file `../../.gotm_env.yaml`, which is created by `gotm_env_init.py`. 

In [2]:
m = Model(name='OSPapa_sensitivity', environ='../gotmtool/.gotm_env.yaml')

Take a look at what are defined in the environment file.

In [3]:
for key in m.environ:
    print('{:>15s}: {}'.format(key, m.environ[key]) )

   gotmdir_code: /Users/qingli/work/A2020_CVMix_in_GOTM/gotm/code
   gotmdir_data: /Users/qingli/work/A2020_CVMix_in_GOTM/gotm/data
  gotmdir_build: /Users/qingli/work/A2020_CVMix_in_GOTM/gotm/build
    gotmdir_exe: /Users/qingli/work/A2020_CVMix_in_GOTM/gotm/exe
    gotmdir_run: /Users/qingli/work/A2020_CVMix_in_GOTM/gotm/run
 gotmdir_figure: /Users/qingli/work/A2020_CVMix_in_GOTM/gotm/figure
   gotmdir_tool: /Users/qingli/work/A2020_CVMix_in_GOTM/gotmtool


## Build the model

In [4]:
%%time
m.build()

CPU times: user 3.12 ms, sys: 8.99 ms, total: 12.1 ms
Wall time: 138 ms


## Configuration
Initialize the GOTM configuration

In [5]:
cfg = m.init_config()

Generating default configuration at '/Users/qingli/work/A2020_CVMix_in_GOTM/gotm/run/OSPapa_sensitivity/gotm.yaml'...
Done!


Update the configuration

In [6]:
# setup
title = 'OSPapa'
depth = 150.0
cfg['title'] = title
cfg['location']['name'] = 'Papa'
cfg['location']['latitude'] = 50.1
cfg['location']['longitude'] = -144.9
cfg['location']['depth'] = depth
cfg['time']['start'] = '2012-03-21 00:00:00'
cfg['time']['stop']  = '2012-04-21 00:00:00'

# output
cfg['output']['gotm_out']['title'] = title
cfg['output']['gotm_out']['time_unit'] = 'hour'
cfg['output']['gotm_out']['time_step'] = 3

# forcing
datadir = os.getcwd()+'/data'
cfg['temperature']['method'] = 'file'
cfg['temperature']['file'] = datadir+'/t_prof.dat'
cfg['salinity']['method'] = 'file'
cfg['salinity']['file'] = datadir+'/s_prof.dat'
cfg['surface']['fluxes']['heat']['method'] = 'file'
cfg['surface']['fluxes']['heat']['file'] = datadir+'/heat_flux.dat'
cfg['surface']['fluxes']['tx']['method'] = 'file'
cfg['surface']['fluxes']['tx']['file'] = datadir+'/momentum_flux.dat'
cfg['surface']['fluxes']['tx']['column'] = 1
cfg['surface']['fluxes']['ty']['method'] = 'file'
cfg['surface']['fluxes']['ty']['file'] = datadir+'/momentum_flux.dat'
cfg['surface']['fluxes']['ty']['column'] = 2
cfg['surface']['u10']['method'] = 'file'
cfg['surface']['u10']['file'] = datadir+'/u10.dat'
cfg['surface']['u10']['column'] = 1
cfg['surface']['v10']['method'] = 'file'
cfg['surface']['v10']['file'] = datadir+'/u10.dat'
cfg['surface']['v10']['column'] = 2
cfg['surface']['swr']['method'] = 'file'
cfg['surface']['swr']['file'] = datadir+'/swr.dat'
# use precipitation - evaporation
cfg['surface']['precip']['method'] = 'file'
cfg['surface']['precip']['scale_factor'] = -2.77778e-7
cfg['surface']['precip']['file'] = datadir+'/emp.dat'
cfg['surface']['sst']['method'] = 'file'
cfg['surface']['sst']['file'] = datadir+'/sst.dat'
cfg['surface']['sss']['method'] = 'file'
cfg['surface']['sss']['file'] = datadir+'/sss.dat'
cfg['waves']['stokes_drift']['us']['method'] = 'file'
cfg['waves']['stokes_drift']['us']['file'] = datadir+'/us_prof.dat'
cfg['waves']['stokes_drift']['us']['column'] = 1
cfg['waves']['stokes_drift']['vs']['method'] = 'file'
cfg['waves']['stokes_drift']['vs']['file'] = datadir+'/us_prof.dat'
cfg['waves']['stokes_drift']['vs']['column'] = 2

# water type (Jerlov II)
cfg['light_extinction']['method'] = 'jerlov-ii'

# EOS
cfg['eq_state']['method'] = 'unesco'
cfg['eq_state']['form'] = 'full-pot'

# configure GLS-C01A
cfg['turbulence']['turb_method'] = 'second_order'
cfg['turbulence']['tke_method'] = 'tke'
cfg['turbulence']['len_scale_method'] = 'gls'
cfg['turbulence']['scnd']['method'] =  'weak_eq_kb_eq'
cfg['turbulence']['scnd']['scnd_coeff'] =  'canuto-a'
cfg['turbulence']['turb_param']['length_lim'] = 'false'
cfg['turbulence']['turb_param']['compute_c3'] = 'true'
cfg['turbulence']['turb_param']['Ri_st'] = 0.25
cfg['turbulence']['generic']['gen_m'] = 1.5 
cfg['turbulence']['generic']['gen_n'] = -1.0
cfg['turbulence']['generic']['gen_p'] = 3.0 
cfg['turbulence']['generic']['cpsi1'] = 1.44
cfg['turbulence']['generic']['cpsi2'] = 1.92
cfg['turbulence']['generic']['cpsi3minus'] = -0.63
cfg['turbulence']['generic']['cpsi3plus'] = 1.0 
cfg['turbulence']['generic']['sig_kpsi'] = 1.0 
cfg['turbulence']['generic']['sig_psi'] = 1.3

## Run the model

Here we run the Papa case in four stages, roughly representing (I) the spring stratification, (II) stable forcing in summer, (III) mixed layer entrainment in fall and winter, and (IV) preconditioning for restratification in winter.
Each stage is run with a combination of vertical resolution of [1., 5., 10.] m and time step of [60, 600, 1800] s.

In [7]:
rundates = {
    '2012-03-21': '2012-05-21',
    '2012-05-21': '2012-08-21',
    '2012-08-21': '2013-01-12',
    '2013-01-12': '2013-03-21',
    }

Set the configurations and labels for each run

In [8]:
cfgs = []
labels = []

In [9]:
for start in rundates.keys():
    end = rundates[start]
    cfg['time']['start'] = start+' 00:00:00'
    cfg['time']['stop']  = end+' 00:00:00'
    print('{} -> {}'.format(start, end))
    for dz in [1., 5., 10.]:
        nlev = int(depth/dz)
        cfg['grid']['nlev']  = nlev
        cfg['output']['gotm_out']['k1_stop'] = nlev+1
        cfg['output']['gotm_out']['k_stop'] = nlev
        for dt in [60.0, 600.0, 1800.0]:
            cfg['time']['dt'] = dt
            label = '_dz{:d}m_dt{:d}s_{:s}_{:s}'.format(int(dz), int(dt), start, end)
            labels.append('GLS-C01A'+label)
            cfg['turbulence']['turb_method'] = 'second_order'
            cfgs.append(copy.deepcopy(cfg))
            labels.append('KPP-CVMix'+label)
            cfg['turbulence']['turb_method'] = 'cvmix'
            cfg['cvmix']['surface_layer']['kpp']['langmuir_method'] = 'none'
            cfgs.append(copy.deepcopy(cfg))
            labels.append('KPPLT-VR12'+label)
            cfg['turbulence']['turb_method'] = 'cvmix'
            cfg['cvmix']['surface_layer']['kpp']['langmuir_method'] = 'lwf16'
            cfgs.append(copy.deepcopy(cfg))
            labels.append('KPPLT-LF17'+label)
            cfg['turbulence']['turb_method'] = 'cvmix'
            cfg['cvmix']['surface_layer']['kpp']['langmuir_method'] = 'lf17'
            cfgs.append(copy.deepcopy(cfg))

2012-03-21 -> 2012-05-21
2012-05-21 -> 2012-08-21
2012-08-21 -> 2013-01-12
2013-01-12 -> 2013-03-21


Run the cases in parallel with 2 processes

In [10]:
%%time
sims = m.run_batch(configs=cfgs, labels=labels, nproc=2)

CPU times: user 369 ms, sys: 110 ms, total: 479 ms
Wall time: 4min 44s
